### Configure LLM & Environment variables

In [ ]:
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:latest",
    # model="deepseek-r1:8b",
    temperature=0.5,
    max_tokens=400
)

load_dotenv('./../.env')


### Installing a Community Tool (Wikipedia)

In [ ]:
%pip install --upgrade --quiet  wikipedia

### Integrating Tool Response with LLM 
### (My POC)

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.runnables import chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import re

wiki_check = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=2))

query = "Which flight crashed in Ahmedabad in 2025 ?"

wiki_result = wiki_check.invoke(query)

input_prompt = ChatPromptTemplate.from_template(

    """
    You are an AI assistant. Use the following context to answer the question correctly.
    Answer in a concise way & do not output the think tags.
    If you don't know the answer, just say "I don't know".

    "context: {context}"
    "question: {question}"

    """
)

# Making remove_think_block as a Runnable in order to chain it
@chain
def remove_think_block(text) -> str:
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL | re.IGNORECASE)

chain = input_prompt | llm | StrOutputParser() | remove_think_block

result = chain.invoke({
    "context": wiki_result,
    "question": query
})

print(f"Result: {result}")

### Creating Custom Tools

In [ ]:
from langchain_core.tools import tool

@tool
def add_numbers(a: int, b: int) -> int:
    "Result: Addition of 2 numbers"
    return a+b

@tool
def subtract_numbers(a: int, b: int) -> int:
    "Result: Subtraction of 2 numbers"
    return a-b

@tool
def multiply_numbers(a: int, b: float) -> float:
    "Result: Multiplication of 2 numbers"
    return a*b


add_numbers.run({"a":133, "b":2212})

In [ ]:
custom_tools = [wiki_check, add_numbers, subtract_numbers, multiply_numbers]

llm_with_tools = llm.bind_tools(tools=custom_tools)

question = "Who is the winner of the ICC Champions Trophy 2025 ?"
# question = "What is the multiplication on 2.5 & 4.8 ?"
# question = "What is the division of 8 & 4 ?"
# question = "I have 10 Dollars, I gave 6 Dollars to mother. How many remain with me ?"
# question = "I have 50K Dollars in my account. I got 56K dollars as bonus. How much is my account balance ?"
# question = "What is the latest Air India flight crash in Ahmedabad ?"

response = llm_with_tools.invoke(question)

print(response.tool_calls)

### Executing the Custom Tool Call & Integrate with LLM

In [ ]:
from langchain_core.messages import HumanMessage

# question = "Who won IPL 2025 ?"
# question = "Latest Air India flight crash ?"
question = "My current salary is 10K, now I got 15 percent hike. What is my new salary ?"
# question = "Who is the winner of the ICC Champions Trophy 2025 ?"


# Get the 1st component -> Question to be asked (HumanMessage)
input_for_llm = [HumanMessage(question + "/no_think")]

# Generate the AI Message with tool call (AIMessage)
ai_message = llm_with_tools.invoke(question)

input_for_llm.append(ai_message)

list_of_tools = {tool.name: tool for tool in custom_tools}

for tool_call in ai_message.tool_calls:
    tool_name = tool_call['name'].lower()
    
    tool_to_execute = list_of_tools[tool_name]
    
    # Generate the context by using the AI Message & the Human Message -> (ToolMessage)
    executed_tool = tool_to_execute.invoke(tool_call)
    
    # Append the last leg before calling the LLM to generate the output
    # HumanMessage, AIMessage & ToolMessage make the input_for_llm complete
    input_for_llm.append(executed_tool)


# Integrating the Tool call (HumanMessage, AIMessage & ToolMessage) to LLM
llm_response = llm_with_tools.invoke(input_for_llm)
print(llm_response.content)